In [39]:
import nltk
import re                                  # library for regular expression operations
import string                              # for string operations
import numpy as np
import pandas as pd

from nltk.corpus import twitter_samples
from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem import PorterStemmer        # module for stemming
from nltk.tokenize import TweetTokenizer   # module for tokenizing strings

nltk.download('twitter_samples', quiet=True)
nltk.download('stopwords', quiet=True)

all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

def remove_hyperlinks_marks_styles(tweet):
    
    # remove old style retweet text "RT"
    new_tweet = re.sub(r'^RT[\s]+', '', tweet)

    # remove hyperlinks
    new_tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', new_tweet)

    # remove hashtags
    # only removing the hash # sign from the word
    new_tweet = re.sub(r'#', '', new_tweet)
    
    return new_tweet

tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)

def tokenize_tweet(tweet):
    
    tweet_tokens = tokenizer.tokenize(tweet)
    
    return tweet_tokens

#Import the english stop words list from NLTK
stopwords_english = stopwords.words('english')

punctuations = string.punctuation

def remove_stopwords_punctuations(tweet_tokens):
    
    tweets_clean = []
    
    for word in tweet_tokens:
        if (word not in stopwords_english and word not in punctuations):
            tweets_clean.append(word)
            
    return tweets_clean
stemmer = PorterStemmer()

def get_stem(tweets_clean):
    
    tweets_stem = []
    
    for word in tweets_clean:
        stem_word = stemmer.stem(word)
        tweets_stem.append(stem_word)
        
    return tweets_stem

def process_tweet(tweet):
    
    processed_tweet = remove_hyperlinks_marks_styles(tweet)
    tweet_tokens = tokenize_tweet(processed_tweet)
    tweets_clean = remove_stopwords_punctuations(tweet_tokens)
    tweets_stem = get_stem(tweets_clean)
    
    return tweets_stem
print('Training NaiveBayes Classification Model....................')
train_pos = all_positive_tweets
train_neg = all_negative_tweets

train_x = train_pos + train_neg

train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))

def create_frequency(tweets, ys):
    
    freq_d = {}
    for tweet, y in zip(tweets, ys): 
        for word in process_tweet(tweet):
            pair = (word, y)
            if pair in freq_d:
                freq_d[pair] += 1 
            else: 
                freq_d[pair] = freq_d.get(pair, 1)
    
    return freq_d

freqs = create_frequency(train_x, train_y)

def train_naive_bayes(freqs, train_x, train_y):
    '''
    Input:
        freqs: dictionary from (word, label) to how often the word appears
        train_x: a list of tweets
        train_y: a list of labels correponding to the tweets (0,1)
    Output:
        logprior: the log prior. 
        loglikelihood: the log likelihood of you Naive bayes equation. 
    '''
    
    loglikelihood = {}
    logprior = 0
    
    # calculate the number of unique words in vocab
    unique_words = set([pair[0] for pair in freqs.keys()])
    V = len(unique_words)
    
    # calculate N_pos and N_neg
    N_pos = N_neg = 0
    for pair in freqs.keys():
        if pair[1]>0: 
            N_pos += freqs[(pair)] 
        else: 
            N_neg += freqs[(pair)]
            
    # TODO: calculate the number of documents (tweets)
    D = train_y.shape[0]
    
    # TODO: calculate D_pos, the number of positive documents (tweets)
    D_pos = sum(train_y)
    
    # TODO: calculate D_neg, the number of negative documents (tweets)
    D_neg = D - sum(train_y)
    
    # TODO: calculate logprior: no of positive tweets / no of negative tweets
    logprior = np.log(D_pos) - np.log(D_neg)
    
    # for each unqiue word
    for word in unique_words:
        
        # get the positive and negative frequency of the word
        freq_pos = freqs.get((word, 1),0)
        freq_neg = freqs.get((word, 0),0)
        
        # calculate the probability that word is positive, and negative
        p_w_pos = (freq_pos + 1) / (N_pos + V)
        p_w_neg = (freq_neg + 1) / (N_neg +V)
        
        # calculate the log likelihood of the word
        loglikelihood[word] = np.log(p_w_pos / p_w_neg)
        
    return logprior, loglikelihood

logprior, loglikelihood = train_naive_bayes(freqs, train_x, train_y) 


def naive_bayes_predict(tweet, logprior, loglikelihood):
    '''
    Input:
        tweet: a string
        logprior: a number
        loglikelihood: a dictionary of words mapping to numbers
    Output:
        p: the sum of all the logliklihoods of each word in the tweet (if found in the dictionary) + logprior (a number)

    '''

    # TODO: process the tweet to get a list of words
    word_l = process_tweet(tweet)
    # TODO: initialize probability to zero
    p = 0

    # TODO: add the logprior
    p += logprior

    for word in word_l:
        if word in loglikelihood: 
            p += loglikelihood[word]
    return p

print('Training of NaiveBayes Classification Model Completed !!!!')
print('Finding Sentiment of the given Tweets..........')
print()
df = pd.read_csv('Tweets.csv') 
df = df.dropna()
sentiment = []
for index, tweet in df.iterrows():
    # print( '%s -> %f' % (tweet, naive_bayes_predict(tweet, logprior, loglikelihood)))
    p = naive_bayes_predict(tweet['text'], logprior, loglikelihood)
    sentiment.append(round(p,1))
df['sentiment'] = sentiment

df.where(df['sentiment'] <0).dropna()

df.to_csv('NaiveResult.csv')

positiveTweets = df.where(df['sentiment'] > 0)
negativeTweets = df.where(df['sentiment'] < 0.0)
neutralTweets = df.where(df['sentiment'] == 0.0)

positiveTweets = positiveTweets.dropna()
negativeTweets = negativeTweets.dropna() 
neutralTweets = neutralTweets.dropna() 
print('---------------------Result-------------------------')
print(f"Positive tweets percentage: {(100*len(positiveTweets)/len(df)):.2f} %")
print(f"Negative tweets percentage: {(100*len(negativeTweets)/len(df)):.2f} %")
print(f"Neutral tweets percentage: {(100*len(neutralTweets)/len(df)):.2f} %")

Training NaiveBayes Classification Model....................
Training of NaiveBayes Classification Model Completed !!!!
Finding Sentiment of the given Tweets..........

---------------------Result-------------------------
Positive tweets percentage: 63.46 %
Negative tweets percentage: 33.69 %
Neutral tweets percentage: 2.84 %
